# Creating Attributes At Run-Time

- `Instances and Setting Function Attributes at Runtime`
    - We saw that we can add to an instance namespace directly at runtime by using `setattr` or
    the dot notation
    
    ```python
      class MyClass:
          language = 'Python'
      obj = MyClass()  # obj.__dict__ -> {}
      obj.version = '3.7' # obj.__dict__ -> {'version' : '3.7'}
     ```
    - What happens if we create a new attribute whose value is a function ?
        - `obj.say_hello = lambda : 'Hello World'`
        then
            - `obj.say_hello` -> `function` not a bound method
            - `obj.say_hello()` -> 'Hello World'
            - but `say_hello` does <u>not</u> have access to the instance namespace. 
    - `Can we create and bind a method to an instance at runtime ?`
        - Yes
        - just need to define a `method` that `binds` the function to the `instance`
        
        ```python
              class MyClass:
                  language = 'Python'
              obj = MyClass()
              from types import MethodType
              # MethodType(function, object)
              # function -> function we want to bind
              # object -> the object to bind to 
              obj.say_hello = MethodType(lambda self: f'Hello {self.language}', obj)
              # say_hello is now a method bound to obj -> 'Hello Python' 
        ```
        - only `obj` has been affected
            - no other instances have that method

In [1]:
class Person:
    pass


In [2]:
p1 = Person()
p2 = Person()

In [3]:
p1.name = 'Alex'

In [4]:
p1.__dict__

{'name': 'Alex'}

In [5]:
p2.__dict__

{}

In [6]:
p1.say_hello = lambda : 'Hello'


In [7]:
p1.say_hello

<function __main__.<lambda>()>

In [9]:
p1.__dict__

{'name': 'Alex', 'say_hello': <function __main__.<lambda>()>}

In [10]:
from types import MethodType

In [11]:
class Person:
    def __init__(self, name):
        self.name = name
        

In [12]:
p1 = Person('Eric')
p2 = Person('Alex')

In [13]:
p1.__dict__, p2.__dict__

({'name': 'Eric'}, {'name': 'Alex'})

In [14]:
def say_hello(self):
    return f'{self.name} says hello'

In [17]:
say_hello(p2)

'Alex says hello'

In [18]:
p1_say_hello = MethodType(say_hello, p1)

In [19]:
p1_say_hello

<bound method say_hello of <__main__.Person object at 0x06AC4970>>

In [20]:
p1.p1_say_hello

AttributeError: 'Person' object has no attribute 'p1_say_hello'

In [21]:
p1.__dict__

{'name': 'Eric'}

In [22]:
p1.say_hello = p1_say_hello

In [23]:
p1.__dict__

{'name': 'Eric',
 'say_hello': <bound method say_hello of <__main__.Person object at 0x06AC4970>>}

In [24]:
hex(id(p1))

'0x6ac4970'

In [25]:
p2.__dict__



{'name': 'Alex'}

In [26]:
p1.say_hello()

'Eric says hello'

In [27]:
getattr(p1, 'say_hello')

<bound method say_hello of <__main__.Person object at 0x06AC4970>>

In [28]:
p1 = Person('Alex')

In [29]:
p1.__dict__

{'name': 'Alex'}

In [30]:
p1.say_hello = MethodType(lambda self: f'{self.name} says hello', p1)

In [31]:
p1.say_hello()

'Alex says hello'

In [32]:
p1 = Person('Alex')
p2 = Person('Eric')

In [35]:
p1.say_hello = MethodType(lambda self: f'{self.name} says hello', p2)

In [36]:
p1.say_hello()

'Eric says hello'

In [37]:
from types import MethodType

class Person:
    def __init__(self, name):
        self.name = name
        
    def register_do_work(self, func):
        setattr(self, '_do_work', MethodType(func, self))
    
    def do_work(self):
        do_work_method = getattr(self, '_do_work', None)
        
        if do_work_method:
            return do_work_method()
        else:
            raise AttributeError('You must first register a do_work method.')
    

In [38]:
math_teacher = Person('Eric')
english_teacher = Person('John')

In [39]:
math_teacher.do_work()

AttributeError: You must first register a do_work method.

In [40]:
def work_math(self):
    return f'{self.name} will teach differential today.'




In [41]:
math_teacher.register_do_work(work_math)

In [42]:
math_teacher.__dict__

{'name': 'Eric',
 '_do_work': <bound method work_math of <__main__.Person object at 0x06ADDB70>>}

In [43]:
hex(id(math_teacher))

'0x6addb70'

In [44]:
math_teacher.do_work()

'Eric will teach differential today.'

In [45]:
def work_english(self):
    return f'{self.name} will analyze Hamlet today'

In [46]:
english_teacher.register_do_work(work_english)

In [47]:
english_teacher.__dict__

{'name': 'John',
 '_do_work': <bound method work_english of <__main__.Person object at 0x06ADD530>>}

In [48]:
english_teacher.do_work()



'John will analyze Hamlet today'

In [49]:
persons = [math_teacher, english_teacher]



In [50]:
for p in persons:
    print(p.do_work())
 

Eric will teach differential today.
John will analyze Hamlet today
